In [3]:
from keras.utils.vis_utils import plot_model
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, LSTM, TimeDistributed, Dense, RepeatVector, CuDNNLSTM, GRU, Bidirectional, Input, CuDNNGRU
from keras.utils import np_utils
from keras.callbacks import TensorBoard
import tensorflow as tf
import os
from keras import backend as K
from keras.models import Model
from keras.layers.core import Dense, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.layers import concatenate
import difflib
from keras.models import load_model
import keras
from keras import losses
import matplotlib.pyplot as plt
import random
from random import choice
import re
import pickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math

os.environ["CUDA_VISIBLE_DEVICES"]="2"
seq_length = 10
test_size = 50000
val_size = 30000
nucleotide = ['0', 'A', 'C', 'G', 'T', '-']
#model5 = load_model('model/seq2seq_nogap_camFam3_1mut.h5')
def decoder(array):
    result = ""
    size = len(array)
    for i in range(size):
        if array[i].tolist() == [0, 0, 0, 0, 0]:
            result=result+"0" 
        elif array[i].tolist() == [1, 0, 0, 0, 0]:
            result=result+"A"
        elif array[i].tolist() == [0, 1, 0, 0, 0]:
            result=result+"C"
        elif array[i].tolist() == [0, 0, 1, 0, 0]:
            result=result+"G"
        elif array[i].tolist() == [0, 0, 0, 1, 0]:
            result=result+"T"
        elif array[i].tolist() == [0, 0, 0, 0, 1]:
            result=result+"-"
    return result

def decoderY(array):
    result = ""    
    if array[i].tolist() == [0, 0, 0, 0, 0]:
            result=result+"0" 
    elif array[i].tolist() == [1, 0, 0, 0, 0]:
        result=result+"A"
    elif array[i].tolist() == [0, 1, 0, 0, 0]:
        result=result+"C"
    elif array[i].tolist() == [0, 0, 1, 0, 0]:
        result=result+"G"
    elif array[i].tolist() == [0, 0, 0, 1, 0]:
        result=result+"T"
    elif array[i].tolist() == [0, 0, 0, 0, 1]:
        result=result+"-"
    return result


def printHitMiss(a,b):
    if a==b:
        return 'Hit'
    else:
        return 'Miss'
    
def accuracy(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count = count+1
    return count/len(a)

def accuracy2(a, b, c):
    count = 0
    count2 =0
    for i in range(len(a)):
        if a[i] != c[i]:
            count2 = count2 +1
        if a[i] != c[i] and b[i]==c[i]:
            count = count+1
    return count/count2

def isMutation(a, b):
    if a!= b:
        print("mutation")


def decode_sequence(input_seq, model, encoder_model, decoder_model):
    nucleotide = ['A', 'C', 'G', 'T', '-']
    # Encode the input as state vectors.
    #print(input_seq[0,0])
    index = 0
    states_value = encoder_model.predict(input_seq)
    #print(len(states_value))
    #print(states_value)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 10))
    target_seq[0][0]= np.hstack((input_seq[0,index], np.array([0,0,0,0,0])))
    #print(target_seq)
    # Populate the first character of target sequence with the start character.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    probability = 1
    
    while not stop_condition:
        index = index +1
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print(output_tokens)
        sampled_nucleotide = nucleotide[np.random.choice(5, 1, p=output_tokens[0, -1, :])[0]]
        
        decoded_seq += sampled_nucleotide
        #print(decoded_sentence)
        # Exit condition: either hit max length
        # or find stop character.
        if (len(decoded_sentence) == seq_length):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, 10))
        temp = np.zeros((5))
        temp[sampled_token_index] = 1
        target_seq[0][0]= np.hstack((input_seq[0, index], temp))
        # target_seq[0, 0, sampled_token_index] = 1
        
        
        # Update states
        states_value = [h, c]

    return decoded_sentence

def get_prob(input_seq, target, model, encoder_model, decoder_model):
    nucleotide = ['A', 'C', 'G', 'T', '-']
    # Encode the input as state vectors.
    index = 0
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, 10))
    target_seq[0][0]= np.hstack((input_seq[0,index], np.array([0,0,0,0,0])))

    stop_condition = False
    decoded_sentence = ''
    probability = []
    
    while not stop_condition:
        index = index +1
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(target[index-1])
        
        probability.append(output_tokens[0, -1, :][sampled_token_index])
        sampled_nucleotide = nucleotide[np.random.choice(5, 1, p=output_tokens[0, -1, :])[0]]
        
        decoded_seq += sampled_nucleotide
        if (len(decoded_sentence) == seq_length):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, 10))
        temp = np.zeros((5))
        temp[sampled_token_index] = 1
        target_seq[0][0]= np.hstack((input_seq[0, index], temp))
        # target_seq[0, 0, sampled_token_index] = 1
        
        
        # Update states
        states_value = [h, c]

    return decoded_sentence, probability

def diffList(a, b):
    count = 0
    length = len(a)
    for i in range(length):
        if a[i] != b[i]:
            count = count+1
    return count

#for seq_index in range(1):
def predict2(X_test, y_test, model, encoder_model, decoder_model, gru=False):
    x_true =[]
    y_hat =[]
    y_true =[]
    probList=[]
    generator_output = []
    productProb = [0]*seq_length
    for seq_index in range(len(X_test)):
        input_seq = X_test[seq_index: seq_index + 1]
        #print(input_seq[0])
        if gru:
            decoded_sentence = decode_gru(input_seq, model, encoder_model, decoder_model)
        else :
            decoded_sentence = decode_sequence(input_seq, model, encoder_model, decoder_model)
        _, prob = get_prob(input_seq, y_test[seq_index], model, encoder_model, decoder_model)
        probList.append(prob)
        prob = [math.log(x) for x in prob]
        productProb = [sum(x) for x in zip(productProb, prob)]
        input_sen = decoder(input_seq[0])
        print(input_sen, ' -> ',
              decoded_sentence, 'True:', decoder(y_test[seq_index]), 
              printHitMiss(decoded_sentence, decoder(y_test[seq_index])), 
              diffList(input_sen, decoded_sentence)
             )
        print(input_sen, ' -> ',
              decoder(y_test[seq_index]), 'True:', decoder(y_test[seq_index]), 
              prob,
              printHitMiss(decoded_sentence, decoder(y_test[seq_index])), 
              diffList(input_sen, decoded_sentence)
             )
        x_true.append(input_sen)
        y_hat.append(decoded_sentence)
        y_true.append(decoder(y_test[seq_index]))
    generator_output.append(input_sen+decoded_seq)
    print(productProb)
    productProb = [x/test_size for x in productProb]
    print("Mean and std of probabilities : {} , {}  ".format(np.mean(probList), np.std(probList)))
    print("Sum of log probabilities : {}".format(productProb))
    print("Percentage of target and prediction being identical: {}".format(accuracy(y_hat, y_true)))
    print("Percentage of training and prediction being identical: {}".format(accuracy(y_hat, x_true)))
    print("Accuracy given mutation happened : {}".format(accuracy2(x_true, y_hat, y_true)))
    np.save('data/hg38_output.npy', generator_output)

def grid_predict(train_size, half, epoch, X_test, y_test):
    model1 = load_model("models/gap_hg38_{}_{}_{}.h5".format(train_size,half,epoch))

    encoder_model1 = load_model("models/Egap_hg38_{}_{}_{}.h5".format(train_size,half, epoch))

    decoder_model1 =load_model("models/Dgap_hg38_{}_{}_{}.h5".format(train_size,half, epoch))

    predict2(X_test, y_test, model1, encoder_model1, decoder_model1, gru=False)
    



# In[3]:
def concat(input1, input2):
    result = []
    for x, y in zip(input1, input2):
        result.append(np.hstack((x, y)))
    
    return np.array(result)

def get_data(trainInd, valInd, testInd):
    X_train=np.load('prepData/X_train_gap_hg38_v3_chr2_size10.npy')[:trainInd]
    X_val=np.load('prepData/X_val_gap_hg38_v3_chr2_size10.npy')[:valInd]
    X_test=np.load('prepData/X_test_gap_hg38_v3_chr2_size10.npy')[:testInd]
    y_train=np.load('prepData/y_train_gap_hg38_v3_chr2_size10.npy')[:trainInd]
    y_val=np.load('prepData/y_val_gap_hg38_v3_chr2_size10.npy')[:valInd]
    y_test=np.load('prepData/y_test_gap_hg38_v3_chr2_size10.npy')[:testInd]

    y_train1 = np.load('prepData/y_train1_gap_hg38_v3_chr2_size10.npy')[:trainInd]
    y_val1 = np.load('prepData/y_val1_gap_hg38_v3_chr2_size10.npy')[:valInd]
    y_test1 = np.load('prepData/y_test1_gap_hg38_v3_chr2_size10.npy')[:testInd]

    y_train1 = concat(X_train, y_train1)
    y_val1 = concat(X_val, y_val1)
    y_test1 = concat(X_test, y_test1)
    return X_test, y_test




train_size = 0
# hidden = [16, 32, 64,128,256,512]
# epoch = [10, 10, 2, 2, 2, 1]
hidden = [512]
epoch = [1]
X_test, y_test = get_data(train_size, val_size, test_size)
for h, e in zip(hidden, epoch):
    print("Train size = {}, hidden_size = {}, epoch = {}".format(train_size, h, e))
    grid_predict(train_size, h, e, X_test, y_test)
    print("The end of Train size = {}, hidden_size = {}, epoch = {}".format(train_size, h, e))

Train size = 0, hidden_size = 512, epoch = 1
[[[0.00201258 0.01226156 0.00356582 0.977424   0.00473603]]]
[[[1.0657682e-03 1.1091941e-02 4.2530606e-03 9.8292130e-01 6.6799053e-04]]]
[[[1.2098398e-03 1.2420672e-02 4.9315188e-03 9.8069972e-01 7.3830347e-04]]]
[[[0.0070936  0.00389863 0.9840794  0.00385827 0.00107013]]]
[[[9.8091549e-01 2.7112628e-03 1.3386744e-02 2.1633632e-03 8.2315155e-04]]]
[[[9.7991198e-01 2.3721172e-03 1.5631458e-02 1.7507970e-03 3.3364649e-04]]]
[[[0.00551531 0.96746683 0.00492661 0.02024466 0.00184654]]]
[[[0.00352275 0.00907388 0.00636095 0.9791334  0.00190897]]]
[[[0.01622919 0.0042205  0.9726397  0.00566547 0.00124518]]]
[[[0.00464759 0.97276515 0.00398202 0.0174369  0.00116845]]]
TTTGAACTGC  ->  TTTGAACTGC True: TTTGAACTGC Hit 0
TTTGAACTGC  ->  TTTGAACTGC True: TTTGAACTGC [-0.02283471338047827, -0.017226220718807077, -0.01948896413513795, -0.01604867322903345, -0.019268973166034622, -0.020292524602855678, -0.033074137655771545, -0.021087356521511696, -0.027741

[[[0.01718118 0.00738017 0.9698629  0.00413333 0.00144245]]]
[[[0.02462736 0.00789423 0.9606415  0.00493348 0.00190339]]]
[[[0.97201914 0.0043866  0.01800868 0.0033953  0.00219026]]]
[[[0.00651901 0.03270002 0.00589688 0.95127517 0.00360898]]]
[[[0.01968011 0.00407659 0.96761346 0.00656322 0.00206662]]]
[[[0.00790624 0.9635005  0.00675108 0.01949852 0.00234366]]]
[[[0.00456964 0.9653427  0.00538202 0.02337778 0.00132794]]]
[[[0.9651553  0.00449897 0.02347684 0.00390031 0.00296859]]]
[[[0.00528369 0.95098907 0.00928133 0.03297602 0.00146994]]]
GGGATGCCAC  ->  GGGATGCCAC True: GGGATGGCAC Miss 0
GGGATGCCAC  ->  GGGATGGCAC True: GGGATGGCAC [-0.050239675617127125, -0.03060058003135538, -0.040153985048641594, -0.028379787522076858, -0.049951910393460905, -0.032922591048431385, -4.998053056339368, -0.04549599304767645, -0.03843137765816942, -0.050138208871042575] Miss 0
[[[0.0198844  0.00407854 0.9685019  0.00375847 0.00377664]]]
[[[0.01796695 0.00672306 0.9690635  0.00487088 0.00137565]]]
[[

[[[0.02038271 0.00747108 0.96551293 0.00417695 0.00245641]]]
[[[0.9779729  0.00362233 0.01253863 0.00311723 0.00274879]]]
[[[0.02286797 0.00514433 0.9672907  0.00340572 0.0012914 ]]]
[[[0.01598048 0.00711642 0.9723208  0.00290133 0.00168107]]]
[[[0.97427124 0.00387834 0.01663192 0.00335901 0.0018594 ]]]
[[[0.01191083 0.7669626  0.01333365 0.20590156 0.00189141]]]
[[[0.23441908 0.00621071 0.74628145 0.00807894 0.00500983]]]
[[[0.0056977  0.9575232  0.00785615 0.02720387 0.00171905]]]
GGGAGGACGC  ->  GGGAGGACGC True: AGGAGGACGC Miss 0
GGGAGGACGC  ->  AGGAGGACGC True: AGGAGGACGC [-3.5044752931706253, -0.028988148697587295, -0.04219454819269887, -0.025042723009941627, -0.03624747076748487, -0.031561918647723224, -0.02688517674484598, -0.27556943865201167, -0.3059090555504365, -0.04287327733160967] Miss 0
[[[0.02739241 0.00441173 0.95946246 0.00356206 0.00517128]]]
[[[0.01604196 0.00803598 0.9703048  0.00376903 0.00184815]]]
[[[0.9766609  0.00443515 0.01382707 0.00299852 0.0020784 ]]]
[[[0.

[[[0.97308177 0.00515208 0.01321337 0.0027725  0.00578035]]]
[[[0.9682395  0.00504159 0.02287272 0.0025147  0.00133158]]]
[[[0.98003894 0.00325899 0.01333923 0.00221879 0.00114412]]]
[[[0.01708376 0.00504838 0.97165257 0.00441801 0.00179738]]]
[[[0.00836702 0.96379715 0.00680799 0.01920718 0.00182077]]]
[[[0.9665806  0.00542757 0.01976032 0.00496548 0.00326611]]]
[[[0.01431207 0.0035199  0.97863436 0.00227682 0.00125695]]]
CCCAAAGCAG  ->  CCCAAAGCAG True: CCCAAAGCAG Hit 0
CCCAAAGCAG  ->  CCCAAAGCAG True: CCCAAAGCAG [-0.04102015986115771, -0.029069635122932744, -0.030905513580056332, -0.027287163781216907, -0.03227581918741367, -0.020162972488055528, -0.02875697937920488, -0.036874429706200224, -0.03399055975876096, -0.021597191963121865] Hit 0
[[[0.00428021 0.96353775 0.00673757 0.02041051 0.00503382]]]
[[[0.00435939 0.97086716 0.005301   0.01814214 0.00133032]]]
[[[0.9747652  0.00389848 0.01426563 0.00231665 0.00475418]]]
[[[0.97098213 0.00450306 0.02130688 0.00221374 0.00099413]]]
[[

[[[0.9701895  0.00471966 0.01823464 0.00413345 0.00272259]]]
[[[1.2235613e-02 4.4903010e-03 9.7988302e-01 2.4506911e-03 9.4038481e-04]]]
[[[0.00358648 0.9747063  0.00514711 0.0152339  0.00132622]]]
[[[0.01336678 0.7723076  0.01602958 0.19579382 0.00250218]]]
[[[0.24616162 0.01163144 0.7257939  0.0090292  0.00738391]]]
[[[0.02167676 0.00634017 0.9664996  0.00384554 0.00163793]]]
[[[0.00596616 0.02756359 0.00631871 0.95711297 0.00303857]]]
[[[0.01710153 0.00437292 0.9724531  0.0043612  0.00171122]]]
[[[0.00675411 0.96310323 0.00628262 0.021722   0.00213805]]]
CAGCCGGTGC  ->  CAGCCGGTGC True: CAGCCGGTGC Hit 0
CAGCCGGTGC  ->  CAGCCGGTGC True: CAGCCGGTGC [-0.03678123580744518, -0.030263853602602787, -0.02032208673296624, -0.02561909218386301, -0.25837239541540463, -0.32048919136771453, -0.03407436663852848, -0.04383385064211426, -0.027933413010870576, -0.03759467171686395] Hit 0
[[[0.9659764  0.00481452 0.01735203 0.00483365 0.0070233 ]]]
[[[0.012579   0.00437517 0.9779269  0.00392365 0.001

...

[[[0.97910327 0.00305925 0.01169674 0.00338656 0.00275421]]]
[[[9.8139787e-01 4.0213098e-03 1.1796642e-02 2.0886876e-03 6.9548091e-04]]]
[[[0.9775749  0.00308032 0.01571456 0.00263914 0.00099119]]]
[[[0.00240816 0.01073959 0.00405502 0.9799717  0.00282551]]]
[[[0.00452155 0.97431016 0.0071945  0.01291113 0.0010626 ]]]
[[[0.00271727 0.00838491 0.00514137 0.9814933  0.00226315]]]
[[[0.00120493 0.01049022 0.00418234 0.98304856 0.00107394]]]
[[[0.97071606 0.00545542 0.01590492 0.00310965 0.00481386]]]
[[[0.01267415 0.00620386 0.97672063 0.00327207 0.00112932]]]
GAAATCTTAG  ->  GAAATCTTAG True: GAAATCTTAG Hit 0
GAAATCTTAG  ->  GAAATCTTAG True: GAAATCTTAG [-0.02954095080977193, -0.02111815969290152, -0.018777328547812257, -0.022680381472239008, -0.020231578277936698, -0.026025586930748124, -0.01868009744479941, -0.01709676205382292, -0.029721274455286002, -0.023554613456428106] Hit 0
[[[0.96735    0.00503717 0.01395634 0.00564513 0.00801133]]]
[[[0.9748011  0.00547792 0.01530306 0.00328036 0

GGCCAGGAGG  ->  GGCCAGGAGG True: GGCCAGGAGG Hit 0
GGCCAGGAGG  ->  GGCCAGGAGG True: GGCCAGGAGG [-0.03852924752069145, -0.03085510492569116, -0.024568574076991673, -0.027512663264905073, -0.032653683160819284, -0.024244375866159545, -0.03260059995324846, -0.024332438908606476, -0.04283308682111883, -0.03087108788244998] Hit 0
[[[0.0221882  0.00457339 0.96182185 0.00686471 0.00455192]]]
[[[0.00492214 0.97304857 0.00543479 0.01517643 0.00141804]]]
[[[0.00420701 0.97287375 0.00529198 0.01651604 0.00111122]]]
[[[0.97020763 0.00360512 0.01928284 0.00305058 0.00385382]]]
[[[0.01364186 0.00521649 0.97755903 0.00187072 0.00171188]]]
[[[0.01663585 0.00760277 0.97020626 0.00434353 0.00121167]]]
[[[0.9762264  0.00343735 0.01506011 0.00357415 0.001702  ]]]
[[[0.03027868 0.00583574 0.95820034 0.00334642 0.00233894]]]
[[[0.01973303 0.00628245 0.9671144  0.00499921 0.00187099]]]
[[[0.00576384 0.02118994 0.00689321 0.9622029  0.00395013]]]
GCCAGGAGGT  ->  GCCAGGAGGT True: GCCAGGAGGT Hit 0
GCCAGGAGGT  ->

TTCTCTCAGC  ->  TTCTCTCAGC True: TTCTCTCAGC Hit 0
TTCTCTCAGC  ->  TTCTCTCAGC True: TTCTCTCAGC [-0.025375616990685065, -0.012310811487656859, -0.03065976465713668, -0.020174406459516094, -0.027270441720938635, -0.021919619073890482, -0.02939208704669685, -0.03870357870950295, -0.031238766411116635, -0.04195060933236465] Hit 0
[[[0.00196887 0.00812336 0.00393007 0.9800986  0.00587908]]]
[[[0.00375502 0.96941805 0.00645874 0.01847304 0.00189513]]]
[[[0.00208014 0.0125214  0.00333721 0.97997576 0.00208547]]]
[[[0.00211491 0.9739077  0.00725116 0.01528679 0.0014395 ]]]
[[[0.00241454 0.01501963 0.00413645 0.975957   0.00247243]]]
[[[0.0026193  0.9707888  0.00882052 0.01615104 0.0016204 ]]]
[[[0.964644   0.00675396 0.0218819  0.00375494 0.00296515]]]
[[[0.01626127 0.00480596 0.97542745 0.00242309 0.00108217]]]
[[[0.00600541 0.96548235 0.00698979 0.0198136  0.00170879]]]
[[[0.00375696 0.00992271 0.00351582 0.97947663 0.00332789]]]
TCTCTCAGCT  ->  TCTCTCAGCT True: TCTCTCAGCT Hit 0
TCTCTCAGCT  -

[[[0.23865485 0.00803626 0.73818946 0.00960312 0.00551619]]]
[[[0.01612932 0.00437853 0.9748839  0.00344811 0.00116017]]]
[[[0.00511833 0.02321096 0.00592896 0.96279615 0.0029456 ]]]
[[[0.9667367  0.00435135 0.02350553 0.00332668 0.00207975]]]
CCTGACGGTA  ->  CCTGACGGTA True: CCTGACGGTA Hit 0
CCTGACGGTA  ->  CCTGACGGTA True: CCTGACGGTA [-0.04067866555576456, -0.030094734363583973, -0.0317024923232793, -0.021298371170339787, -0.022270612006631652, -0.282826247889867, -0.30355476802348647, -0.02543687764322555, -0.03791357013352352, -0.03382913259927128] Hit 0
[[[0.00331399 0.96908736 0.00576157 0.0157065  0.00613056]]]
[[[0.00293554 0.0097017  0.00529157 0.9803596  0.00171155]]]
[[[0.01151193 0.00381047 0.98090714 0.00202083 0.00174973]]]
[[[0.9789844  0.00440863 0.01160044 0.00364715 0.00135934]]]
[[[0.01489451 0.7577756  0.01594444 0.20937736 0.00200801]]]
[[[0.2453538  0.00777578 0.73097944 0.00979302 0.00609801]]]
[[[0.01641413 0.00457183 0.97439414 0.00332352 0.00129642]]]
[[[0.005

KeyboardInterrupt: 